In [1]:
pip install nltk scikit-learn streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 802.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00


In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
# Define pairs of patterns and responses
pairs = [
    # Greetings
    [
        r"hi|hello|hey",
        ["Hello! Welcome to the Udaipur Travel Guide. How can I assist you today?", "Hi there! How can I help you with your Udaipur travel plans?"]
    ],
    # Tourist Locations
    [
        r"what are the best tourist places in Udaipur|top attractions in Udaipur|places to visit in Udaipur",
        ["Udaipur is famous for its beautiful lakes and palaces. Some must-visit places are:\n"
         "1. City Palace\n"
         "2. Lake Pichola\n"
         "3. Jag Mandir\n"
         "4. Saheliyon Ki Bari\n"
         "5. Bagore Ki Haveli\n"
         "6. Monsoon Palace\n"
         "7. Fateh Sagar Lake\n"
         "Let me know if you want details about any specific place!"]
    ],
    [
        r"tell me about (.*)",
        [
            "Sure! Here's some information about %1:\n"
            "1. **City Palace**: A majestic palace complex with museums, courtyards, and stunning views of Lake Pichola.\n"
            "2. **Lake Pichola**: A picturesque lake with boat rides and views of the surrounding palaces.\n"
            "3. **Jag Mandir**: A beautiful island palace in Lake Pichola.\n"
            "4. **Saheliyon Ki Bari**: A historic garden with fountains and lush greenery.\n"
            "5. **Bagore Ki Haveli**: A restored haveli showcasing royal artifacts and cultural performances.\n"
            "6. **Monsoon Palace**: A hilltop palace offering panoramic views of Udaipur.\n"
            "7. **Fateh Sagar Lake**: A serene lake surrounded by hills and gardens.\n"
            "Would you like more details about any of these?"
        ]
    ],
    # Budget and Prices
    [
        r"what is the budget for Udaipur|how much does it cost to visit Udaipur|prices in Udaipur",
        ["The budget for Udaipur depends on your travel style:\n"
         "1. **Budget Travel**: ₹1000-₹2000 per day (budget hotels, local food, public transport).\n"
         "2. **Mid-Range Travel**: ₹3000-₹5000 per day (3-star hotels, restaurants, private taxis).\n"
         "3. **Luxury Travel**: ₹7000+ per day (5-star hotels, fine dining, private tours).\n"
         "Let me know if you need help with specific expenses!"]
    ],
    [
        r"how much does a boat ride cost in Lake Pichola|boat ride price",
        ["The boat ride in Lake Pichola costs around ₹400-₹600 per person for a basic ride. Private boat rides can cost more."]
    ],
    # Restaurants
    [
        r"best restaurants in Udaipur|where to eat in Udaipur|top restaurants",
        ["Udaipur has some amazing restaurants:\n"
         "1. **Ambrai**: Offers lakeside dining with a view of City Palace.\n"
         "2. **Upre**: Rooftop restaurant with a view of Lake Pichola.\n"
         "3. **Charcoal By Carlsson**: Known for its continental cuisine.\n"
         "4. **Millets of Mewar**: Great for healthy and organic food.\n"
         "5. **Jheel's Ginger Coffee Bar**: Perfect for coffee lovers.\n"
         "Let me know if you want more recommendations!"]
    ],
    # Weather
    [
        r"what is the weather like in Udaipur|best time to visit Udaipur|Udaipur weather",
        ["Udaipur has a semi-arid climate. Here's the best time to visit:\n"
         "1. **October to March**: Pleasant weather, ideal for sightseeing (15°C to 25°C).\n"
         "2. **April to June**: Hot and dry (up to 40°C).\n"
         "3. **July to September**: Monsoon season with moderate rainfall.\n"
         "Plan your trip accordingly!"]
    ],
    # Hotels
    [
        r"best hotels in Udaipur|where to stay in Udaipur|hotels in Udaipur",
        ["Here are some great hotels in Udaipur:\n"
         "1. **The Oberoi Udaivilas**: A luxury hotel with stunning architecture.\n"
         "2. **Taj Lake Palace**: A iconic hotel located in the middle of Lake Pichola.\n"
         "3. **Trident Udaipur**: A mid-range option with beautiful lake views.\n"
         "4. **Jagat Niwas Palace Hotel**: A heritage hotel with a royal feel.\n"
         "5. **Treebo Trend Krishna Niwas**: A budget-friendly option with good amenities.\n"
         "Let me know if you need help with bookings!"]
    ],
    # Souvenirs and Gifts
    [
        r"best souvenirs to buy in Udaipur|what to buy in Udaipur|gifts from Udaipur",
        ["Udaipur is known for its unique souvenirs and gifts:\n"
         "1. **Miniature Paintings**: Handmade paintings depicting Rajasthani culture.\n"
         "2. **Silver Jewelry**: Exquisite designs crafted by local artisans.\n"
         "3. **Pichwai Paintings**: Traditional cloth paintings of Lord Krishna.\n"
         "4. **Rajasthani Handicrafts**: Pottery, textiles, and wooden items.\n"
         "5. **Bandhani Sarees**: Traditional tie-and-dye sarees.\n"
         "You can find these at Hathi Pol Bazaar and Bada Bazaar."]
    ],
    # Temples
    [
        r"famous temples in Udaipur|temples to visit in Udaipur|religious places in Udaipur",
        ["Udaipur has several beautiful temples:\n"
         "1. **Jagdish Temple**: A 17th-century temple dedicated to Lord Vishnu.\n"
         "2. **Eklingji Temple**: A historic temple complex dedicated to Lord Shiva.\n"
         "3. **Neemach Mata Temple**: Located on a hilltop with panoramic views.\n"
         "4. **Sas Bahu Temple**: Known for its intricate carvings.\n"
         "5. **Shri Manshapurna Karni Mata Temple**: A serene temple near the ropeway.\n"
         "Let me know if you need more details!"]
    ],
    # Transportation
    [
        r"how to travel in Udaipur|transport in Udaipur|local transport options",
        ["Here are some ways to get around Udaipur:\n"
         "1. **Auto-rickshaws**: Affordable and widely available.\n"
         "2. **Taxis**: Private taxis can be hired for full-day tours.\n"
         "3. **Buses**: Local buses connect major areas.\n"
         "4. **Bike Rentals**: Scooters and bikes can be rented for self-exploration.\n"
         "5. **Boat Rides**: Available on Lake Pichola and Fateh Sagar Lake.\n"
         "Let me know if you need help with transportation!"]
    ],
    # Goodbye
    [
        r"quit|bye|goodbye",
        ["Thank you for using the Udaipur Travel Guide. Have a great trip!", "Goodbye! Safe travels!"]
    ],
    # Default Response
    [
        r"default",
        ["I'm sorry, I didn't understand that. Can you please rephrase your question?"]
    ]
]
